In [1]:
%matplotlib inline
import numpy as np;
from matplotlib import pyplot as plt

$\LaTeX \text{ commands here}
\newcommand{\R}{\mathbb{R}}
\newcommand{\im}{\text{im}\,}
\newcommand{\norm}[1]{||#1||}
\newcommand{\inner}[1]{\langle #1 \rangle}
\newcommand{\span}{\mathrm{span}}
\newcommand{\proj}{\mathrm{proj}}
\newcommand{\OPT}{\mathrm{OPT}}
$

<hr style="border: 5px solid black">

**Georgia Tech, CS 4540**

# L14:  Linear Programming Duality

Jake Abernethy & Benjamin Bray

*Thursday, October 17, 2019*

## Reading

* Tim Roughgarden, Stanford CS261
    * ["Lecture 8:  Linear Programming Duality I"](http://theory.stanford.edu/~tim/w16/l/l8.pdf)
    * ["Lecture 9:  Linear Programming Duality II"](http://theory.stanford.edu/~tim/w16/l/l9.pdf) (Optional)
* Jim Burke, University of Washington MATH 407
    * ["Section 1:  Linear Programming"](https://sites.math.washington.edu/~burke/crs/407/notes/section1.pdf)
    * ["Section 2:  Simplex Algorithm"](https://sites.math.washington.edu/~burke/crs/407/notes/section2.pdf)

$$
\begin{align}
\max_{x \in \R^n} &\quad c^T x \\
\text{such that}  &\quad Ax \leq b & (A \in \R^{m \times n}) \\
                  &\quad  x \geq 0
\end{align}
$$

### Farkas Lemma

We will work towards proving the following Lemma:

<div style="padding:20px;margin:20px;border: 1px solid black">
<b>Lemma.</b> (Farkas)  Let $A \in \R^{m \times n}$ and $b \in \R^m$.  Then exactly one of the following two statements is true:

1.  There exists $x \in \R^n$ such that $Ax = b$ and $x \geq 0$.
2.  There exists $y \in \R^m$ such that $y^T A \geq 0$ and $b^T y < 0$.
</div>

### Problem:  Farkas Lemma, Part 1

<div style="padding:20px;margin:20px;border: 1px solid black">
<b>Problem:</b> Prove that 
$$
(\exists\, x \geq 0 \text{ s.t. } Ax = b) \iff b \in \mathrm{cone}(a_1, \dots, a_n)
$$
where $\mathrm{cone}\{ a_1, \dots, a_n \}$ is the set of all conic combinations of the columns of $A$.
</div>

### Solution:  Farkas Lemma, Part 1

* $(\implies)$ Suppose there is $x \geq 0$ such that $A x = b$ and $x \geq 0$.  Since $A x$ is a linear combination of the columns of $A$ with coefficients from $x$, which has nonnegative entries, $A x = b \in \mathrm{cone}\{ a_1,\dots,a_n \}$ by definition.
* $(\impliedby)$ Similarly, if $b \in \mathrm{cone}(a_1,\dots,a_n)$, then there are coefficients $x_1,\dots,x_n \geq 0$ such that $b = \sum_{k=1}^n x_1 a_1 \in \R^m$.  Assemble the coefficients into a vector $x = (x_1,\dots,x_n) \in \R^n$.  Then $A x = b$.

### Problem:  Farkas Lemma, Part 2

<div style="padding:20px;margin:20px;border: 1px solid black">
<b>Problem:</b> (not 1 implies 2) Show that:

* If there does not exist $x \geq 0$ such that $Ax = b$...
* then there exists $y \in \R^m$ such that $y^T A \geq 0$ and $b^T y < 0$.
</div>

> *Hint:*  Use the separating hyperplane theorem, where one set is $\mathrm{cone}(a_1,\dots,a_k)$.

### Problem:  Farkas Lemma, Part 2

* Assume there is no $x \geq 0$ such that $Ax = b$.  By the previous problem, then $b \notin \mathrm{cone}(a_1,\dots,a_n)$.
* Notice that $y^T A = [ y^T a_1, y^T a_2, \dots, y^T a_n ] \in \R^{1 \times n}$.  So, $y^T A \geq 0$ is equivalent to:
$$
y^T A \geq 0
\iff
y^T a_j \geq 0 \quad \forall\, j=1,\dots,n
$$
* Interpret $y$ as the normal vector to a hyperplane.  So, we want to show that there is a hyperplane which separates $b \in \R^m$ from the columns of $A$.
* By assumption, $b$ is disjoint from $\mathrm{cone}(a_1,\dots,a_n)$ and they are both closed sets.  Use the separating hyperplane theorem to find $y \in \R^m$ separating $b$ from the cone.  Since the columns of $A$ belong to the cone, we are done!

## Example: Solving shortest path using Linear Program

Imagine you have a directed graph graph $G = (V,E)$ where each edge $e \in E$ has some "travel time" $c_e$ associated with it. You have a source $t \in V$ and a sink $s \in V$. Formulate a linear program that solves the "continuous flow" version of shortest path! Does not need to be in standard form.

### Recall:  Linear Program in Standard Form

<div style="padding:20px;margin:20px;border: 1px solid black">
\begin{align}
\max_{x \in \R^n} &\quad c^T x \\
\text{such that}  &\quad Ax \leq b & \text{(only $\leq$ constraints)} \\
                  &\quad  x \geq 0 & \text{(variables nonnegative)}
\end{align}
</div>

* Decision variables $x \in \R^n$
* Linear objective $c^T x$ for $c \in \R^n$
* Constraint matrix $A \in \R^{m \times n}$ and vector $b \in \R^m$.  Each row corresponds to a constraint.

### Recall:  Geometric Intuition

* Linear constraints correspond to half-spaces, so the **feasible region** is a polytope.
* Level sets of the objective are parallel hyperplanes, all orthogonal to the coefficient vector $c$.
* The optimum is the farthest vector in the feasible region along the direction $c$.
* For bounded feasible regions, a solution always exists at a vertex.

![](images/l6-lpexample.png)

### Recall:  Combinations of Constraints

If $y = (y_1,\dots,y_m) \geq 0$, then $y^T A$ is a nonnegative linear combination of the constraints (the rows of $A$).  Look for combinations of constraints that **dominate** the objective,
    $$
    y^T A = \sum_{k=1}^m y_k a_k^T \geq c^T \implies c^T x \leq y^T A x \leq y^T b
    $$
    
This is an upper bound on the value of the objective function!
    
> *Note:  People usually write $A^T y \geq c$, which is equivalent.*

### Recall:  Dual Linear Program

The **dual linear program** searches for the *best* or *tightest* upper bound by looking at all possible combinations of constraints which dominate the objective:

<div style="padding:20px;margin:20px;border: 1px solid black">
$$
\begin{align}
\min_{y \in \R^m} &\quad y^T b \\
\text{such that}  &\quad y^T A \geq c^T & \text{(dominates objective)} \\
                  &\quad  y \geq 0 & \text{(nonnegative combinations)}
\end{align}
$$
</div>

* The $y=(y_1,\dots,y_m) \in \R^m$ are called **dual variables**.
    * (in contrast to **primal variables** $x \in \R^n$)
* The new constraint matrix is $A^T \in \R^{n \times m}$.
    * One dual constraint per primal variable (if primal was in standard form).

### Weak Duality

Using the "tightest upper bound" interpretation of the dual linear program, it is clear that **weak duality** holds:

<div style="padding:20px; margin:20px; border:1px solid black">
<b>Theorem:</b> The optimal value for any (maximization) linear program is no larger than the value of the corresponding dual program, that is, $$\OPT(Primal) \leq \OPT(Dual)$$
</div>

* The value of every primal-feasible point is no larger than the value of any dual-feasible point.
* If the optimal value of $Primal$ is unbounded, then $Dual$ is infeasible.
* If the optimal value of $Dual$ is unbounded, then $Primal$ is infeasible.

### Problem:  Checking Optimality

Recall that a feasible solution $x \in \R^n$ to *Primal* is called **optimal** if no other feasible solution has a higher objective value.

<div style="padding:20px;margin:20px;border: 1px solid black">
<b>Problem:</b>  Suppose $x \in \R^n$ is primal-feasible and $y \in \R^m$ is dual-feasible.  Prove that if $c^T x = y^T b$, then both $x$ and $y$ are optimal in their respective linear programs.  (<em>Hint:</em>  Weak Duality)
</div>

**Solution:** Assume towards contradiction that $x' \in \R^n$ has a higher objective value, $c^T x' > c^T x$.  Then $c^T x' > y^T b$.  But since $y$ is dual feasible, this contradicts weak duality!

### Recall:  Activity Analysis Problem

A company has $m$ different resources which can be used to manufacture $n$ different products.

* $x = (x_1,\dots,x_n) \in \R^n$ is the amount of each product manufactured
* $r = (r_1,\dots,r_m) \in \R^m$ is the supply of each resource
* $c = (c_1,\dots,c_n) \in \R^n$ is the profit generated by each product
* $A = [a_{ij}] \in \R^{m \times n}$, where $a_{ij}$ is the amount of resource $i$ needed to make product $j$

<div style="padding:10px;margin:20px;border: 1px solid black">
$$\max_{x \in \R^n} \quad c^T x \quad
\text{s.t.} \quad A x \leq r \quad\text{and}\quad x \geq 0$$
</div>

### Problem:  Interpreting the Dual

<div style="padding:20px;margin:20px;border: 1px solid black">
<b>Problem:</b>  Write down the dual linear program for the activity analysis problem.  What "units" do the dual variables need have?  How can they be interpreted?
</div>

> If it helps, assume the amount of each resource available is measured in *kilograms*, and our profit is measured in *euros* €.  Maybe we're manufacturing rope, so the amount of each product is measured in *meters*.